In [1]:
import cv2
import numpy as np 
import os
import random 
from tqdm import tqdm
from keras.layers.convolutional import Conv2D
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import MaxPooling2D, Dropout, Flatten, Dense

Using TensorFlow backend.
C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Miniconda3\lib\site-packages\tensorflow\python\framework\dtype

In [2]:
def crop_and_save(img, path = "C:\\Users\\Pankratz\\Desktop\\Jupyter_lab2\\"): 
    
    assert img == 'check.jpg' or img == 'cross.jpg', "значение img должно быть 'check.jpg' или 'cross.jpg' "
    
    img_path = os.path.join(path, img)
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) # читаем изображение сразу чёрно-белым
    if img == 'cross.jpg': 
        image = image[20:727, 60:727] # обрезаем ненужные места со скана 
        max_x = 17
        max_y = 18
        new_path = 'C:\\Users\\Pankratz\\Desktop\\Jupyter_lab2\\images\\training data\\0' # сразу кодируем X как 0
    elif img == 'check.jpg': 
        image = image[25:735, 32:736] # обрезаем ненужные места со скана
        max_x = 18 # в листах с крестиками и галочками разное число символов в каждой строке
        max_y = 17
        new_path = 'C:\\Users\\Pankratz\\Desktop\\Jupyter_lab2\\images\\training data\\1' # сразу кодируем V как 1
    
    size = 32 
    final_wide = max_x*size # рассчитываем во сколько раз уменьшим картинку
    r = float(final_wide) / image.shape[1]
    dim = (final_wide, int(image.shape[0] * r))

    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    # gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(resized, (5, 5), 1) 
    result = cv2.threshold(blurred, 235, 255, cv2.THRESH_BINARY)[1] #преобразованное изображение
    
    i = 0
    x0 = 0
    y0 = 0
    count_x = 0
    
    while count_x < max_x: # цикл по строкам
        count_y = 0
        x0 = 0
        while count_y < max_y: # цикл по столбцам
            cropped = result[x0:x0+size, y0:y0+size]
            final_path = os.path.join(new_path, "{0}.jpg".format(i))
            cv2.imwrite(final_path, cropped)
            x0 += size
            i += 1
            count_y +=1
        y0 += size
        count_x += 1
    
    return i 
    
    

In [3]:
def load_data(train_array, train_answers, my_path = 'C:\\Users\\Pankratz\\Desktop\\Jupyter_lab2\\images'):
    classes = ['0', '1']
    directory1 = os.path.join(my_path, 'training data')
    for c in classes:
        directory_classes = os.path.join(directory1, c)
        for pic in tqdm(os.listdir(directory_classes)):
            stock = cv2.imread(os.path.join(directory_classes, pic), cv2.IMREAD_GRAYSCALE) 
            train_array.append(stock / 255)
            train_answers.append(c)
    
    
    #images = [cv2.imread(file) for file in glob.glob("path/to/files/*.png")]
    
    #shfl = list(zip(train_array, train_answers))
    #random.shuffle(shfl)
    #train_array, train_answers = zip(*shfl)
    
    

In [4]:
#models 

classes = ['0', '1']

def one_layer_dense():
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu', name = 'input'))
    model.add(Dense(len(classes), activation = 'softmax', name = 'output'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    return model

def many_layers_dense():
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(1024, activation = 'relu', name = 'input'))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation = 'relu'))
    model.add(Dense(len(classes), activation = 'softmax', name = 'output'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    return model

def conv_model():
    model = Sequential()
    model.add(Conv2D(filters = 8, kernel_size = (3,3), activation = 'relu', input_shape = (32, 32, 1)))
    model.add(MaxPooling2D((3,3), strides = 1))
    model.add(Conv2D(filters = 6, kernel_size = (3,3), activation = 'relu', input_shape = (32, 32, 1)))
    model.add(MaxPooling2D((3,3), strides = 1))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(len(classes), activation = 'softmax', name = 'output'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['categorical_accuracy'])
    return model
    

    





In [5]:
def predict_answers(model, X_pred, num_pred, conv = False): 
    
    if conv == True:
        X_pred = X_pred.reshape(X_pred.shape[0], 32, 32, 1 ) 
        
    arr_pred = np.array(random.choices(X_pred, k = num_pred)) #случайно перемешиваем поданный массив
    predictions = model.predict(arr_pred) #генерируем предсказания

    for p in range(len(predictions)):
        if predictions[p][0] > 0.95: #0.95 - порог уверенности в том что картинка принадлежит к типу
            print('cross \n')
        elif predictions[p][1] > 0.95:
            print('check \n')
        else:
            print('i dunno \n')
        cv2.imshow('w', arr_pred[p]) #
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [6]:
#one layer model

import keras


X_train = []
y_train = []
X_test = []
y_test = []

a = crop_and_save("check.jpg") 
b = crop_and_save("cross.jpg") 
sum = a+b

load_data(X_train, y_train)

X_train = (np.array(X_train))
y_train = np.array(y_train)
y_train_encoded = (keras.utils.to_categorical(y_train))
#print(X_train.shape, y_train_encoded.shape)
#print(y_train_encoded)


X_pred1 = X_train[:2] #выделяем немного данных для демонстрации model.predict
y_pred1 = y_train_encoded[:2] #с разных концов упорядоченного датасета, чтобы попадались картинки обоих типов
X_pred2 = X_train[-2:]
y_pred2 = y_train_encoded[-2:] 

X_test = X_train[2:100] #выделяем примерно шестую часть данных для тестового сета 
y_test = y_train_encoded[2:100]

X_train = X_train[100:-2]
y_train_encoded = y_train_encoded[100:-2]

model1 = one_layer_dense()
model1.fit(X_train, y_train_encoded, validation_split = 0.2, batch_size = 17, epochs = 18)

results = model1.evaluate(X_test, y_test, batch_size = 20)
print('\n test loss, test acc: {0}'.format(results))

X_pred = np.concatenate((X_pred1, X_pred2))
num_pred = 4 #количество делаемых предсказаний
# y_test = np.concatenate((y_test1, y_test2))
# ответы не нужны для предсказаний
print('Предсказания модели: \n')
predict_answers(model1, X_pred, 4) 


100%|███████████████████████████████████████████████████████████████████████████████| 306/306 [00:02<00:00, 142.17it/s]



Train on 408 samples, validate on 102 samples
Epoch 1/18
408/408 [==============================] - 0s 1ms/step - loss: 0.4227 - categorical_accuracy: 0.8652 - val_loss: 0.0196 - val_categorical_accuracy: 0.9902
Epoch 2/18
408/408 [==============================] - 0s 563us/step - loss: 0.0267 - categorical_accuracy: 0.9926 - val_loss: 0.0431 - val_categorical_accuracy: 0.9804
Epoch 3/18
408/408 [==============================] - 0s 568us/step - loss: 0.0168 - categorical_accuracy: 0.9926 - val_loss: 0.0250 - val_categorical_accuracy: 0.9804
Epoch 4/18
408/408 [==============================] - 0s 570us/step - loss: 0.0146 - categorical_accuracy: 0.9951 - val_loss: 0.0102 - val_categorical_accuracy: 1.0000
Epoch 5/18
408/408 [==============================] - 0s 547us/step - loss: 0.0089 - categorical_accuracy: 0.9975 - val_loss: 0.0083 - val_categorical_accuracy: 1.0000
Epoch 6/18
408/408 [==============================] - 0s 529us/step - loss: 0.0051 - categorical_accuracy: 1.0000 -

In [ ]:
#many layers model

import keras


X_train = []
y_train = []
X_test = []
y_test = []

a = crop_and_save("check.jpg") 
b = crop_and_save("cross.jpg") 
sum = a+b

load_data(X_train, y_train)

X_train = (np.array(X_train))
y_train = np.array(y_train)
y_train_encoded = (keras.utils.to_categorical(y_train))
#print(X_train.shape, y_train_encoded.shape)
#print(y_train_encoded)


X_pred1 = X_train[:2] #выделяем немного данных для демонстрации model.predict
y_pred1 = y_train_encoded[:2] #с разных концов упорядоченного датасета, чтобы попадались картинки обоих типов
X_pred2 = X_train[-2:]
y_pred2 = y_train_encoded[-2:] 

X_test = X_train[2:100] #выделяем примерно шестую часть данных для тестового сета 
y_test = y_train_encoded[2:100]

X_train = X_train[100:-2]
y_train_encoded = y_train_encoded[100:-2]

model2 = many_layers_dense()
model2.fit(X_train, y_train_encoded, validation_split = 0.2, batch_size = 17, epochs = 18)

results = model2.evaluate(X_test, y_test, batch_size = 20)
print('\n test loss, test acc: {0}'.format(results))

X_pred = np.concatenate((X_pred1, X_pred2))
num_pred = 4 #количество делаемых предсказаний
# y_test = np.concatenate((y_test1, y_test2))
# ответы не нужны для предсказаний
print('Предсказания модели: \n')
predict_answers(model2, X_pred, 4) 

In [ ]:
#convolutional model

import keras


X_train = []
y_train = []
X_test = []
y_test = []

a = crop_and_save("check.jpg") 
b = crop_and_save("cross.jpg") 
sum = a+b

load_data(X_train, y_train)

X_train = (np.array(X_train))
y_train = np.array(y_train)
y_train_encoded = (keras.utils.to_categorical(y_train))
#print(X_train.shape, y_train_encoded.shape)
#print(y_train_encoded)


X_pred1 = X_train[:2] #выделяем немного данных для демонстрации model.predict
y_pred1 = y_train_encoded[:2] #с разных концов упорядоченного датасета, чтобы попадались картинки обоих типов
X_pred2 = X_train[-2:]
y_pred2 = y_train_encoded[-2:] 

X_test = X_train[2:100] #выделяем примерно шестую часть данных для тестового сета 
y_test = y_train_encoded[2:100]

X_train = X_train[100:-2]
y_train_encoded = y_train_encoded[100:-2]

X_train = X_train.reshape(X_train.shape[0], 32, 32, 1)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 1)

model3 = conv_model()
model3.fit(X_train, y_train_encoded, validation_split = 0.2, batch_size = 17, epochs = 18)

results = model3.evaluate(X_test, y_test, batch_size = 20)
print('\n test loss, test acc: {0}'.format(results))

X_pred = np.concatenate((X_pred1, X_pred2))
num_pred = 4 #количество делаемых предсказаний
print('Предсказания модели: \n')
predict_answers(model3, X_pred, 4, conv = True) 